<a href="https://colab.research.google.com/github/mohammad-zoubi/Generative_modelling_for_melanoma_detection/blob/main/image_processing/cut_image_out.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import
import zipfile
import cv2
import albumentations as A
import numpy as np
from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow
import random
import os
from zipfile import ZipFile
import pandas as pd
import seaborn as sns
import shutil
from PIL import ImageFile
from io import BytesIO

In [ ]:
zipped_file_path = "/content/drive/MyDrive/ExJobb/ISIC256x256/ISIC256_train_unprocessed.zip"
mod_zipped_file_path = "/content/drive/MyDrive/ExJobb/ISIC256x256/"
csv_file_path = "/content/drive/MyDrive/ExJobb/ISIC256x256/bias_pseudoannotations_real_train_ISIC20.csv"
SAM_zipped_file_path = "/content/drive/MyDrive/ExJobb/SAM256_unprocessed.zip"
SAM_unprocessed_imgs = "/content/drive/MyDrive/ExJobb/Sahlgrenska/SAM_unprocessed"

In [ ]:
def hair_removal(img, kern, intensity):
    # src = cv2.imread(path_in)
    src = img
    # Convert the original image to grayscale
    grayScale = cv2.cvtColor( src, cv2.COLOR_RGB2GRAY )
    
    # Kernel for the morphological filtering
    kernel = cv2.getStructuringElement(1,(kern,kern))

    # Perform the blackHat filtering on the grayscale image to find the 
    # hair countours
    blackhat = cv2.morphologyEx(grayScale, cv2.MORPH_BLACKHAT, kernel)
  
    # intensify the hair countours in preparation for the inpainting 
    # algorithm
    ret,thresh2 = cv2.threshold(blackhat,intensity,255,cv2.THRESH_BINARY)
    
    # inpaint the original image depending on the mask
    dst = cv2.inpaint(src,thresh2,1,cv2.INPAINT_TELEA)
    
    return dst

In [ ]:
### Specify the number of images
for m in range(692, 1344+1):
    
    img = cv2.imread(img)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    _,thresh = cv2.threshold(gray,1,255,cv2.THRESH_BINARY)
    # cv2_imshow(gray)

    threshold = 80
    shadow_threshold = 0.025*gray_cropped_image.shape[0]

    histo = np.histogram(gray,100,  [30,255])
    for i in range(histo[0].shape[0]-5, 0, -1):
      if histo[0][i-1] < np.mean([histo[0][i], histo[0][i+1], histo[0][i+2], histo[0][i+3], histo[0][i+4]]):
        threshold_skin = histo[1][i-1]
        break;
    threshold_skin = threshold_skin - 30

    # Finding where the pixels are above the threshold
    for j in range(gray.shape[1]):
      find = False
      for i in range(gray.shape[0]):
        if gray[i][j] >= threshold:
          y_init = j
          find = True
          break
      if find:
        break
    # Finding the cordinates from all 4 sides in the picture
    for j in range(gray.shape[1]-1,0,-1):
      find = False
      for i in range(gray.shape[0]-1,0,-1):
        if gray[i][j] >= threshold:
          y_fin = j
          find = True
          break
      if find:
        break

    for i in range(gray.shape[1]):
      find = False
      for j in range(gray.shape[0]):
        if gray[i][j] >= threshold:
          x_init = i
          find = True
          break
      if find:
        break

    for i in range(gray.shape[1]-1,0,-1):
      find = False
      for j in range(gray.shape[0]-1,0,-1):
        if gray[i][j] >= threshold:
          x_fin = i
          find = True
          break
      if find:
        break

    # crop_dims = np.array([x_init, y_init, x_fin, y_fin])
    # Cropping the image so that only what's above the threshold is included
    # for both the original and gray image
    cropped_image = img[y_init:y_fin, x_init:x_fin]
    gray_cropped_image = gray[y_init:y_fin, x_init:x_fin]
    good_pixels = []
    bad_pixels = []
    for j in range(gray_cropped_image.shape[1]):
      for i in range(gray_cropped_image.shape[0]):
        if gray_cropped_image[i][j] < 30:
          bad_pixels.append(np.array([i,j]))
        elif gray_cropped_image[i][j] > threshold_skin:
          good_pixels.append([i,j])
    good_pixels = np.array(good_pixels)
    bad_pixels = np.array(bad_pixels)
    # I'm a little cinfused as to what happens here?
    bad_list = np.random.randint(bad_pixels.shape[0], size=bad_pixels.shape[0])
    good_list = np.random.randint(good_pixels.shape[0], size=good_pixels.shape[0])

    bad_pixel_list = bad_pixels[bad_list]
    good_pixel_list = good_pixels[good_list]
    # cropped_image[bad_pixel_list] = cropped_image[good_pixel_list]
    
    temp = 0
    #Replacing "bad" pixels with good ones
    # 2nd quadrant
    for i in range(int(np.floor(gray_cropped_image.shape[0]/2))):
        for j in range(int(np.floor(gray_cropped_image.shape[1]/2))):
            
            if gray_cropped_image[i,j] < threshold or gray_cropped_image[i,j] < gray_cropped_image[i+1,j+1]:
                rand = np.random.randint(0, good_pixel_list.shape[0])
                cropped_image[i,j,:] = cropped_image[good_pixel_list[rand][0], good_pixel_list[rand][1], :] 
            elif temp > shadow_threshold:
                temp = 0
                break
            else :
                temp = temp +1

            

    # 1st quadrant
    for i in range(int(np.floor(gray_cropped_image.shape[0]/2))):
        for j in range(gray_cropped_image.shape[1]-3, int(np.floor(gray_cropped_image.shape[1]/2)-3), -1):
            # if gray_cropped_image[i,j] > 2*threshold:
            #     break
            if gray_cropped_image[i,j] < threshold or gray_cropped_image[i,j] < gray_cropped_image[i+1,j-1]:
                rand = np.random.randint(0, good_pixel_list.shape[0])
                cropped_image[i,j,:] = cropped_image[good_pixel_list[rand][0], good_pixel_list[rand][1], :] 
            elif temp > shadow_threshold:
                temp = 0
                break
            else :
                temp = temp +1

    # 3rd quadrant
    for i in range(gray_cropped_image.shape[0]-2, int(np.floor(gray_cropped_image.shape[0]/2)-3), -1):
        for j in range(int(np.floor(gray_cropped_image.shape[1]/2))):
            # if gray_cropped_image[i,j] > 2*threshold:
            #     break
            if gray_cropped_image[i,j] < threshold or gray_cropped_image[i,j] < gray_cropped_image[i-1,j+1]:
                rand = np.random.randint(0, good_pixel_list.shape[0])
                cropped_image[i,j,:] = cropped_image[good_pixel_list[rand][0], good_pixel_list[rand][1], :] 
            elif temp > shadow_threshold:
                temp = 0
                break
            else :
                temp = temp +1

    # 4th quadrant
    for i in range(gray_cropped_image.shape[0]-2, int(np.floor(gray_cropped_image.shape[0]/2)-3), -1):
        for j in range(gray_cropped_image.shape[1]-2, int(np.floor(gray_cropped_image.shape[1]/2)-3), -1):
            # if gray_cropped_image[i,j] > 2*threshold:
            #     break
            if gray_cropped_image[i,j] < threshold or gray_cropped_image[i,j] < gray_cropped_image[i-1,j-1]:
                rand = np.random.randint(0, good_pixel_list.shape[0])
                cropped_image[i,j,:] = cropped_image[good_pixel_list[rand][0], good_pixel_list[rand][1], :] 
            elif temp > shadow_threshold:
                temp = 0
                break
            else :
                temp = temp +1



    # path_out = os.path.join('/content/drive/MyDrive/pre_processed_images(different sizes)/', "{:04d}".format(m)+ '_derm_01.jpeg')
    path_out = os.path.join('/content/drive/MyDrive/ELLA_Eivor1.0_DERM_dataset.zip (Unzipped Files)/preprocessed/', "{:04d}".format(m)+ '_derm_01.jpeg')
    
    cv2.imwrite(path_out, cropped_image)

In [ ]:
zip_name = "processed_imgs.zip"
processed_img_zip = zipfile.ZipFile(os.path.join(mod_zipped_file_path,zip_name), "w")
# idx = 0
with ZipFile(zipped_file_path, "r") as zip_ref:    
    # Get list of files names in zip
    list_of_files = zip_ref.namelist()
    # print(list_of_files)
   # Iterate over the list of file names in given list & print them
    for idx in range(pseudo_labels.index.shape[0]):
        if idx == pseudo_labels.index[-1]:
          break
      
        # print(elem)
        # print(os.path.join(zipped_file_path, elem))
        # ext = os.path.splitext(elem)[-1]  # Get extension of elem
        # name = os.path.splitext(elem)[0][12:] # Name of image 
        # if ext == ".jpg":
        # Read data in case extension is ".jpg"
        in_bytes = zip_ref.read(os.path.join('train/train/',pseudo_labels.image_name[idx]))

        # Decode bytes to image.
        img = cv2.imdecode(np.frombuffer(in_bytes, np.uint8), cv2.IMREAD_COLOR)

        
        biases = False
        hair = pseudo_labels.hair_dense[idx] == 1
        hair = pseudo_labels.hair_medium[idx] == 1
        hair = pseudo_labels.hair_short[idx] == 1

        frame = pseudo_labels.black_frame[idx] == 1

        # check hair label
        if hair == True or frame == True:
          biases = True
        
        if frame == True:
          # print("frame")
          img = frame_removal(img)
          # pass image to frame removal function

        img = hair_removal(img, kern=10, intensity=5)

        img_encoded = cv2.imencode('.jpg', img)[1].tobytes()  # bytes class
    
        # 'repair' image from byte array
        nparr = np.frombuffer(img_encoded, np.byte)
    
        # encoded_image = cv2.imdecode(, cv2.IMREAD_COLOR)
        processed_img_zip.writestr(pseudo_labels.image_name[idx], nparr)
        # idx += 1
    processed_img_zip.close()

In [ ]:
zip_name = "SAMtmp_processed_images.zip"
processed_img_zip = zipfile.ZipFile(os.path.join(mod_zipped_file_path,zip_name), "w")
with ZipFile(SAM_zipped_file_path, "r") as zip_ref:    
    # Get list of files names in zip
    list_of_files = zip_ref.namelist()
    for idx in range(1 ,len(list_of_files)): # list of files has all the files in zip includin the folders
      ext = os.path.splitext(list_of_files[idx])[-1]
      if ext == ".jpg":
        in_bytes = zip_ref.read(list_of_files[idx])
        img = cv2.imdecode(np.frombuffer(in_bytes, np.uint8), cv2.IMREAD_COLOR)
        # img = hair_removal(img, kern=10, intensity=5)
        img_encoded = cv2.imencode('.jpg', img)[1].tobytes()  # bytes class
        nparr = np.frombuffer(img_encoded, np.byte)
        processed_img_zip.writestr(list_of_files[idx][7:], nparr)
    processed_img_zip.close()

In [ ]:
img_path = os.path.join(SAM_unprocessed_imgs, os.listdir(SAM_unprocessed_imgs)[10])

In [ ]:
import cv

ERROR:root:Error disabling cv.imshow().
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/google/colab/_import_hooks/_cv2.py", line 86, in load_module
    cv_module.imshow,
AttributeError: module 'cv' has no attribute 'imshow'


In [ ]:
img = cv2.imread(img_path,0)
img = cv2.medianBlur(img,5)
cimg = cv2.cvtColor(img,cv2.COLOR_GRAY2BGR)

circles = cv2.HoughCircles(img,cv2.HOUGH_GRADIENT,1,200)

circles = np.uint16(np.around(circles))

cropSize = (1000, 1000) # <- added: what size do you want to extract

for i in circles[0,:]:
    #draw outer circle
    cv2.circle(cimg,(i[0],i[1]),i[2],(0,255,0),2)
    #draw center of circle
    cv2.circle(cimg,(i[0],i[1]),2,(0,0,255),3)

    #crop ROI around circle...?
    # make sure the bounds won't under-/overflow
    cropCoords = (max(0, i[1]-cropSize[0]//2),min(img.shape[0], i[1]+cropSize[0]//2),
                  max(0, i[0]-cropSize[1]//2),min(img.shape[1], i[0]+cropSize[1]//2)) 
    crop_cimg = cimg[cropCoords[0]:cropCoords[1],
                     cropCoords[2]:cropCoords[3]] 


cv2_imshow(crop_cimg)


NameError: ignored

In [ ]:
cv2_imshow(cv2.HoughCircles(image=tmp,
                 method=cv2.HOUGH_GRADIENT,
                 dp=1,
                 minDist=1,
                 maxRadius=-1, 
                 param1=50,param2=30))

NameError: ignored

In [ ]:
tmp = cv2.Canny(img,10,10)